## Korzystanie z API Open AI z użyciem _function calling_

Function calling to mechanizm, który pozwala LLMom na wywoływanie zdefiniowanych funkcji w odpowiedzi na zapytania użytkownika. Dzięki temu modele mogą wykonywać bardziej złożone operacje, takie jak pobieranie danych z zewnętrznych API, przetwarzanie informacji czy wykonywanie obliczeń. Mechanizm ten umożliwia integrację modeli LLM z różnymi narzędziami i usługami, co zwiększa ich funkcjonalność i zakres zastosowań.

Więcej informacji, w tym na temat konfiguracji, użycia oraz przykłady, znajdziecie w [dokumentacji](https://platform.openai.com/docs/guides/function-calling).

In [ ]:
import requests
import json
from helpers import parse_nested_json # funkcja która parsuje zagnieżdżone jsony, aby potem było można je wypisać w dobrze sformatowany sposób

## Funkcja pobierająca współrzędne geograficzne dla danej nazwy

In [ ]:
def get_geolocation(location):
    """
    Pobiera współrzędne geograficzne oraz dane lokalizacyjne.

    Parametry:
    location (str): Nazwa lokalizacji, dla której chcemy uzyskać współrzędne geograficzne.

    Zwraca:
    dict: Dane lokalizacyjne w formacie JSON.
    """
    # Definiowanie endpointu i parametrów dla API OpenStreetMap Nominatim
    geocode_endpoint = "https://nominatim.openstreetmap.org/search"
    geocode_params = {
        "q": location,
        "format": "json"
    }

    # Nagłówek User-Agent jest wymagany do korzystania z tego API
    headers = {
        "User-Agent": "Python script"
    }

    # Wykonanie zapytania
    geocode_response = requests.get(geocode_endpoint, params=geocode_params, headers=headers)

    # Wyciągnięcie współrzędnych geograficznych
    geocode_data = geocode_response.json()
    
    # Zwrócenie tylko nazwy miejsca, szerokości i długości geograficznej
    simplified_data = {
        "name": geocode_data[0]["display_name"],
        "latitude": geocode_data[0]["lat"],
        "longitude": geocode_data[0]["lon"]
    }
    return simplified_data

# Przykład użycia funkcji
geolocation_data = get_geolocation("Poznań")
print(json.dumps(geolocation_data, indent=4))


## Funkcja pobierająca informacje o pogodzie dla podanych współrzędnych geograficznych

In [ ]:
def get_wind_direction(degrees):
    """
    Konwertuje kierunek wiatru z stopni na nazwy kierunków.

    Parametry:
    degrees (float): Kierunek wiatru w stopniach.

    Zwraca:
    str: Nazwa kierunku wiatru.
    """
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
                  'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    index = int((degrees + 11.25) // 22.5) % 16
    return directions[index]

def get_current_weather(latitude, longitude):
    """
    Pobiera aktualne dane pogodowe dla podanych współrzędnych geograficznych.

    Parametry:
    latitude (float): Szerokość geograficzna.
    longitude (float): Długość geograficzna.

    Zwraca:
    dict: Dane pogodowe w formacie JSON.
    """
    # Zdefiniowanie endpointu i parametrów dla Open Meteo API
    weather_endpoint = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True
    }

    # Wykonanie requestu
    weather_response = requests.get(weather_endpoint, params=weather_params)

    # Wyciągnięcie informacji o pogodzie
    weather_data = weather_response.json()

    simplified_weather = {
        "temperature": f"{weather_data['current_weather']['temperature']} °C",
        "wind_speed": f"{weather_data['current_weather']['windspeed']} km/h",
        "wind_direction": get_wind_direction(weather_data['current_weather']['winddirection']),
        "is_day": "day" if weather_data['current_weather']['is_day'] else "night"
    }

    return simplified_weather

# Przykład użycia funkcji
current_weather = get_current_weather(geolocation_data['latitude'], geolocation_data['longitude'])
print(json.dumps(current_weather, indent=4))

## Przygotowanie informacji do modelu o tym, jakie funkcje mamy dostępne

In [ ]:
# lista opisów dostępnych funkcji, które pozwalamy modelowi wywołać
# Każdy zawiera nazwę funkcji, opis jej działania oraz parametry, które funkcja przyjmuje.
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_geolocation",
            "description": "Pobiera współrzędne geograficzne oraz dane lokalizacyjne.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Nazwa lokalizacji, dla której chcemy uzyskać współrzędne geograficzne."
                    }
                },
                "required": ["location"],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Pobiera aktualne dane pogodowe dla podanych współrzędnych geograficznych.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "Szerokość geograficzna lokalizacji dla której pobierana jest pogoda."
                    },
                    "longitude": {
                        "type": "number",
                        "description": "Długość geograficzna lokalizacji dla której pobierana jest pogoda."
                    }
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

## Funkcja pomocnicza, która wykonywa wskazaną przez model funkcję - o ile ten o to poprosi

In [ ]:
# funkcja obsługuje wywołania funkcji na podstawie nazwy funkcji i argumentów.
def handle_function_call(function_name, arguments):
    print(f"<function_call> Function: {function_name}, Arguments: {arguments}")

    arguments = json.loads(arguments)

    if function_name == "get_geolocation":
        res = get_geolocation(arguments['location'])
        return json.dumps(res)
    elif function_name == "get_current_weather":
        res = get_current_weather(arguments['latitude'], arguments['longitude'])
        return json.dumps(res)
    else:
        return {"error": "Unknown function"}

## Funkcja pomocnicza, która wyciąga z odpowiedzi modelu które funkcje i z jakimi argumentami mają zostać wykonane, i następnie je wykonuje z użyciem powyższej funkcji

In [ ]:
def execute_tool_calls(response, messages):
    """
    Wykonuje funkcje zlecone przez model i aktualizuje historię wiadomości.

    Parametry:
    response (openai.types.chat.chat_completion.ChatCompletion): Odpowiedź od modelu zawierająca zlecenia funkcji.
    messages (list): Historia wiadomości.

    Zwraca:
    list: Zaktualizowana historia wiadomości.
    """

    # dodajemy odpowiedź, w tym ew. prośby o wywołanie funkcji, do historii wiadomości
    messages.append(response.choices[0].message.model_dump())

    for tool_call in response.choices[0].message.tool_calls:
        # pobieramy nazwę funkcji i argumenty z odpowiedzi od modelu
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        # wywołanie funkcji na podstawie pobranej nazwy funkcji i argumentów
        result = handle_function_call(name, json.dumps(args))

        # dodajemy wynik funkcji do historii wiadomości
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })

    return messages


#### Przesłanie początkowego zapytania do modelu

In [ ]:
from openai import OpenAI
import os

# Inicjalizacja klienta
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key) # konfiguracja połączenia z API

messages = [
    {"role": "system", "content": "Jesteś pomocnym asystentem."},
    {"role": "user", "content": "Opisz jaka jest pogoda w Poznaniu. Czy powinienem wychodzić na spacer w stroju plażowym i okularach przeciwsłonecznych?"},
]

# Przykład zapytania do GPT-4o z function callingiem
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools, # przekazujemy listę dostępnych funkcji
    tool_choice="auto",
    temperature=0.7,
    max_tokens=16000
)


#### Wykonanie funkcji zleconych przez model

W tym przypadku wiemy, że model zleci wykonanie funkcji pobrania lokalizacji dla Poznania, ponieważ sami określiliśmy jego wejściowe zapytanie. Gdybyśmy oprogramowywali system, w którym użytkownik wprowadza dowolny tekst, nie moglibyśmy tego zakładać - użytkownik może zadać pytanie, do odpowiedzi na które dostępne dla modelu funkcje mogą nie być przydatne.

In [ ]:
# i wywołujemy, aby obsłużyć zlecenia modelu odnośnie wykonania funkcji
messages = execute_tool_calls(response, messages)

In [ ]:
# podejrzyjmy co teraz jest w messages
parsed_data = parse_nested_json(messages)

print(json.dumps(parsed_data, indent=4, ensure_ascii=False))

#### Kolejny request do modelu - zwrócenie wyniku funkcji

In [ ]:
# Kolejny request, z wynikiem wykonanej funkcji
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0.7,
    max_tokens=16000
)

#### Obsługa kolejnej funkcji zleconej przez model

In [ ]:
# ponownie obsługujemy zlecone przez model wykonania funkcji
messages = execute_tool_calls(response, messages)

In [ ]:
# podejrzyjmy co teraz jest w messages
parsed_data = parse_nested_json(messages)

print(json.dumps(parsed_data, indent=4, ensure_ascii=False))

#### Kolejny request do modelu - zwrócenie wyniku kolejnej funkcji

In [ ]:
# Kolejny request, z wynikiem kolejnej funkcji
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0.7,
    max_tokens=16000
)

#### Ostateczny wynik

In [ ]:
# zobaczmy co model nam ostatecznie odpowiedział
print(response.choices[0].message.content)

# A teraz wszystko jako jedna funkcja

In [ ]:
def process_user_command(user_command):
    """
    Przetwarza polecenie użytkownika z użyciem GPT-4o oraz funkcji get_geolocation i get_current_weather.

    Parametry:
    user_command (str): Polecenie użytkownika.

    Zwraca:
    str: Odpowiedź na polecenie użytkownika.
    """

    messages = [
        {"role": "system", "content": "Jesteś pomocnym asystentem."},
        {"role": "user", "content": user_command}
    ]
    
    while True:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.7,
            max_tokens=16000
        )
        
        message = response.choices[0].message
        
        # Jeśli model żąda wykonania funkcji, wykonujemy je i aktualizujemy historię wiadomości
        # (nastąpi kolejne wykonanie pętli zawierające request do modelu, w którym przekażemy wyniki wykonania funkcji)
        if getattr(message, "tool_calls", None):
            messages = execute_tool_calls(response, messages)
        else:
            # Gdy nie ma już żądań funkcji, zwracamy ostateczną odpowiedź modelu.
            return message.content, messages


In [ ]:
# wykonanie funkcji
user_command = "Podaj aktualną pogodę dla Poznania. Czy mam założyć kurtkę?"
# user_command = "Podaj aktualną pogodę dla współrzędnych: 52 st. 24 min. N 16 st. 55 min. E."
# user_command = "ile to jest 2+2?"

response_content, messages = process_user_command(user_command)
print(response_content)

In [ ]:
# zobaczmy co jest w holistrii wiadomości
parsed_data = parse_nested_json(messages)

print(json.dumps(parsed_data, indent=4, ensure_ascii=False))